IMPORTS

In [3]:
!pip install ijson
import pandas as pd
import os
from tqdm import tqdm
import ijson
import json
import gzip
import ijson
import json
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from google.colab import drive
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import PowerTransformer
import nltk
from nltk.corpus import stopwords
drive.mount('/content/drive')

ads_dataset_path = '/content/drive/MyDrive/CS491MLMODEL/us/us/ads.json'
ads_downsized_path = '/content/drive/MyDrive/CS491MLMODEL/us/us/ads_downsized.json'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


DATA DOWNSIZING

In [11]:
json_item_heuristic = 3000000

def downsize_data(num_of_items, input_dataset_path, output_dataset_path):
    counter = 0
    with open(input_dataset_path, 'r') as input_file:
        parser = ijson.items(input_file, 'item')

        with open(output_dataset_path, 'a') as outfile:  # Open output file once
            outfile.write('[\n')
            # Use tqdm to create a progress bar
            for item in tqdm(parser, total=int(num_of_items), unit=" items"):
                if counter > num_of_items:
                    break
                counter += 1
                json.dump(item, outfile)
                if counter > num_of_items:
                  outfile.write('\n')  # Add a newline between items
                else:
                  outfile.write(',\n')  # Add a newline between items

            outfile.write(']\n')
            outfile.close()

downsize_data(json_item_heuristic, ads_dataset_path, ads_downsized_path)

767264 items [05:44, 2227.86 items/s]                        


DATA DUPLICATE REMOVAL

In [ ]:
import json
ads_non_duplicate_path = '/content/drive/MyDrive/CS491MLMODEL/us/us/ads_non_duplicate.json'

def remove_duplicates(filename, column, output_filename):
    json_item_heuristic = 3000
    first_item = True
    removed_content = 0
    unique_values = set()

    with open(filename, 'rb') as file:
        items = ijson.items(file, 'item')

        with open(output_filename, 'w') as outfile:
            outfile.write('[\n')
            for item in tqdm(items, total=int(json_item_heuristic), unit=" items"):
                if 'ad_creative_bodies' in item:
                    column_value = tuple(item['ad_creative_bodies'])

                    if (column_value not in unique_values) and (len(column_value) == 1):
                        unique_values.add(column_value)
                        if first_item:
                            json.dump(item, outfile)
                            first_item = False
                        else:
                            outfile.write(',\n')
                            json.dump(item, outfile)
                    else:
                        removed_content += 1
                else:
                    removed_content += 1
                    pass
            outfile.write(']\n')
    print(f"Amount of ads that have been filtered: {removed_content}")


remove_duplicates(ads_downsized_path, 'a', ads_non_duplicate_path)

JSON TO CSV CONVERSION

In [ ]:
import pandas as pd

ads_non_duplicate_path = '/content/drive/MyDrive/CS491MLMODEL/us/us/ads_non_duplicate.json'
ads_dataframe = '/content/drive/MyDrive/CS491MLMODEL/us/us/ads_dataframe.csv'

with open(ads_non_duplicate_path, 'r') as file:
    data = json.load(file)

processed_data = []
for item in data:
    selected_attributes = {'ad_creation_time': item['ad_creation_time'],
                           'ad_creative_bodies': item['ad_creative_bodies'],
                           'currency': item['currency'],
                           'impressions': item['impressions'],
                           'spend': item['spend']}
    processed_data.append(selected_attributes)

df = pd.DataFrame(processed_data)

df.to_csv(ads_dataframe, index=False)

DATAFRAME INFORMATION

In [ ]:
df = pd.read_csv(ads_dataframe)
print(df.dtypes)
df.head()

CONVERT INTO DATAFRAME

In [ ]:
import ast
new_df = pd.DataFrame()

new_df['ad_creation_time'] = pd.to_datetime(df['ad_creation_time'])
new_df['ad_creative_bodies'] = df['ad_creative_bodies'].apply(lambda x: ast.literal_eval(x)[0] if pd.notnull(x) else x).astype(str)


# Extract 'lower_bound' and 'upper_bound' values if they exist, otherwise use NaN
new_df['impressions_lower'] = df['impressions'].apply(lambda x: ast.literal_eval(x).get('lower_bound', None)).astype("float")
new_df['impressions_upper'] = df['impressions'].apply(lambda x: ast.literal_eval(x).get('upper_bound', None)).astype("float")

new_df['spend_lower'] = df['spend'].apply(lambda x: ast.literal_eval(x).get('lower_bound', None)).astype("float")
new_df['spend_upper'] = df['spend'].apply(lambda x: ast.literal_eval(x).get('upper_bound', None)).astype("float")

print(new_df.dtypes)

new_df.head()

MERGE COLUMNS AND SORT ACCORDING TO DATE

In [ ]:
ads_dataframe = pd.DataFrame()
ads_dataframe['ad_creation_time'] = new_df['ad_creation_time']
ads_dataframe['ad_creative_bodies'] = new_df['ad_creative_bodies']
ads_dataframe['cpi'] = (
    (new_df['spend_lower'].fillna(new_df['spend_upper']) / 2 +
     new_df['spend_upper'].fillna(new_df['spend_lower']) / 2) /
    (new_df['impressions_lower'].fillna(new_df['impressions_upper']) / 2 +
     new_df['impressions_upper'].fillna(new_df['impressions_lower']) / 2)
)

ads_dataframe = ads_dataframe.sort_values(by='ad_creation_time')
ads_dataframe

VECTORIZE TEXTUAL CONTENT

In [ ]:
def tokenize_and_remove_stopwords(text):
    tokens = word_tokenize(text)
    tokens = [word.lower() for word in tokens if word.isalpha() and word.lower() not in stop_words]
    return tokens

def average_word_vectors(words, model, vocabulary, num_features):
    feature_vector = np.zeros((num_features,), dtype="float64")
    nwords = 0.

    for word in words:
        if word in vocabulary:
            nwords = nwords + 1.
            feature_vector = np.add(feature_vector, model.wv[word])

    if nwords:
        feature_vector = np.divide(feature_vector, nwords)

    return feature_vector

def generate_doc_vectors(docs, model, num_features):
    doc_vectors = [average_word_vectors(doc, model, model.wv.index_to_key, num_features) for doc in docs]
    return np.array(doc_vectors)


nltk.download('punkt')
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

# Determine the split index
split_index = int(0.8 * len(ads_dataframe))

train_data = ads_dataframe.iloc[:split_index]
test_data = ads_dataframe.iloc[split_index:]

X_train = train_data['ad_creative_bodies'].astype(str)
y_train = train_data['cpi']

X_test = test_data['ad_creative_bodies'].astype(str)
y_test = test_data['cpi']

# Tokenization and stopword removal
tokenized_text_train = X_train.apply(tokenize_and_remove_stopwords)
tokenized_text_test = X_test.apply(tokenize_and_remove_stopwords)

# Word2Vec model training
word2vec_model = Word2Vec(sentences=tokenized_text_train, vector_size=100, window=5, min_count=1, workers=4)

# Generate document vectors
X_train_word2vec = generate_doc_vectors(tokenized_text_train, word2vec_model, 100)
X_test_word2vec = generate_doc_vectors(tokenized_text_test, word2vec_model, 100)

#word2vec_model.save("word2vec_model.model")

TRAIN ML MODEL

In [ ]:
# Model training and evaluation
gb_regressor = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
gb_regressor.fit(X_train_word2vec, y_train)

predictions = gb_regressor.predict(X_test_word2vec)

mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print(f"Mean Squared Error (Original): {mse}")